In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from pandas_datareader import data as web
import yfinance as yf
import seaborn as sb
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

# 1. Calculo Valuation baseado em Fluxo de Caixa Descontado

Fluxo de Caixa Livre da Firma - É o resultado da empresa depois de pagar todo necessario para manter e expandir a firma (incluindo pagamento de credores e acionistas).

Para calcular o Fluxo de Caixa Livre precisamos calcular o NOPAT (Resultado Operacional Liquido) com as seguintes informações: 
* EBIT (Obter essa informação da seção resultados do Status Invest)
* Aliquota de IR da empresa (Obter essa informação do documento Demonstrações financeiras da empresa, disponível no canal de relação com o investidor)
* Depreciação e amortização (Obter essa informação da seção resultados do Status Invest - O ideal é que se tire uma média dos ultimos anos, para evitar distorções)
* CAPEX (Obter essa informação no fundamentei. Também é interessante verificar se não há distorções significativas no capex)
* Variação Capital de Giro (Obter essa informação no Oceans 14. Obter a diferença entre o ano atual e o ano anterior)

Podemos obter essas informações no Status Invest

In [16]:
data = pd.read_csv("Data\\statusinvest-busca-avancada.csv", delimiter=";")
data_WEG = data[data.TICKER == "WEGE3"]
data_WEG

,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,...,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
611,WEGE3,"37,14","1,85","31,37","9,67","5,18","31,56","18,80","15,70","26,20",...,"0,54","0,45","1,05","25,71","34,19","248.320.983,14","3,84","1,18","0,78","155.888.390.445,72"


In [30]:
def Calculo_NOPAT(ticker, ebit, aliquota_ir):
    nopat = ebit * (1 - aliquota_ir)
    return round(nopat,2)

def calculo_fluxo_caixa_livre(ticker, nopat, depreciacao_amortizacao, capex, variacao_capital_de_giro):
    fluxo_de_caixa = nopat + depreciacao_amortizacao - capex + variacao_capital_de_giro
    return fluxo_de_caixa    

In [29]:
nopat_ticker = Calculo_NOPAT("CAML3", 662470000, 0.34) 
fluxo_caixa_livre_ticker = calculo_fluxo_caixa_livre("CAML3", nopat_camil, 122455000, 236000000, -176000000) 
nopat_ticker, fluxo_caixa_livre_ticker

(437230200.0, 147685200.0)

O segundo passo consiste em calcular a taxa de desconto. 
Conforme proposto anteriormente, queremos trazer o fluxo de caixa dos próximos anos a valor presente e para fazer isso precisamos utilizar uma taxa. 
Seria como se nós estivessemos calculando o fluxo de caixa da empresa no futuro, exemplo, suponha que daqui a 10 anos o fluxo de caixa da empresa seja 100 mil reais. Queremos descobrir, usando a taxa de desconto, quanto valeria hoje a quantia de 100 mil reais daqui a 10 anos. 

Essa taxa de desconto é chamada de Custo médio ponderado de capital (WAC). 

Considere que a empresa pode se financiar de duas formas:
* Credores (pegando dinheiro no banco) - Nessa modalidade a empresa paga juros.
* Capital proprio (captando $ de acionistas) - Nessa modalidade a empresa não paga juros, mas tem que dividir seu lucro com os acionistas.

Nas duas formas a empresa tem um custo ao captar dinheiro. O que o custo médio ponderado de capital faz é justamente mostrar qual o custo médio dessa captação de capital.

Para calcular o WACC nós precisamos das seguintes informações: 
* We (Percentual de capital próprio) - Encontrado no Status Invest na seção de balanço patrimonial (Patrimonio Liquido / Passivo Total)
* Ke (Custo de capital proprio)
* Taxa livre de risco (Pode ser a taxa do EUA ou a Selic)
* Premio de mercado (Consultar website do Damodaran. Esse é um valor tabelado).
* Beta (O quanto um ativo tende a variar em comparação com o IBOV - consultar no site do Investing.com)
* Wd (Percentual de capital de terceiros) - Encontrado no Status Invest na seção de balanço patrimonial ( 1 - (Patrimonio Liquido / Passivo Total))
* Kd (Custo de capital de terceiros) - É o quanto a empresa paga de juros para pegar dinheiro de terceiros. Esse indicador não é divulgado pelas empresas, porém podemos pegar essa informação nas demonstrações contábeis. Veremos o quanto ela paga de juros nas suas emissões de debentures (ferramenta que as empresas usam para captar capital de terceiros).

In [46]:
def Calculo_WACC(we, taxa_livre_risco, premio_mercado, beta, wd):
    ke = taxa_livre_risco + (premio_mercado * beta)
    kd = taxa_livre_risco - 0.0010 + 0.0155
    calculo_wacc = (we*ke)+(wd*kd)
    return round(calculo_wacc,3)

wacc_ticker = Calculo_WACC(0.44, 0.045, 0.0459, 0.379, 0.56)
wacc_ticker

0.061

O ultimo passo consiste em determinar a taxa de crescimendo da empresa. Essa etapa é crucial.
O fluxo de caixa que calculamos até o momento foi para o primeiro ano. Teremos agora que projetar o fluxo de caixa para os proximos anos. 

Estimar a taxa de crescimento é algo bem subjetivo e depende da experiencia de cada analista.

In [47]:
tx_crescimento_3anos  = 0.05 
tx_crescimento_perenidade = 0.015  #Esse valor nunca pode ser maior do que a propria economia. Seja cauteloso e use um valor baixo.

Ano1 = fluxo_caixa_livre_ticker
Ano2 = Ano1 * (1 + tx_crescimento_3anos)
Ano3 = Ano2 * (1 + tx_crescimento_3anos)
perenidade = Ano3*(1+tx_crescimento_perenidade)/(wacc_ticker - tx_crescimento_perenidade)

In [48]:
#Trazendo dados a valor presente: 
Ano1_presente = Ano1/(1+wacc_ticker)
Ano2_presente = Ano2/((1+wacc_ticker)**2)
Ano3_presente = Ano3/((1+wacc_ticker)**3)
perenidade_presente = perenidade/((1+wacc_ticker)**4)

In [49]:
#Fluxo de caixa descontado
DFC = Ano1_presente + Ano2_presente + Ano3_presente + perenidade_presente
num_acoes = 360000000
preco_ideal = round(DFC/num_acoes,2)
preco_ideal

9.02